In [ ]:
import pandas as pd
import numpy as np
import spacy
import re
import os
import urllib
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans

In [ ]:
dataset_url = 'https://huggingface.co/datasets/aniketiyer15/training_data/resolve/main/ner.csv'
dataset_path = 'ner.csv'

if not os.path.isfile(dataset_path):
    urllib.request.urlretrieve(dataset_url, dataset_path)
    print('Download complete.')
else:
    print('Dataset already downloaded.')

Download complete.


*Hi there, these code change ner.csv ---> SpaCy traning data format*

*To avoid errors, I replaced some ***puctuations*** and ***signs*** in ner.csv with some other characters (saved in ner_modified.csv):*

**?** becomes **#** ;

**( )** and **[ ]** becomes **{ }** ;

**+** becomes **|** ;

**$** becomes **@** ;

*(yes, email adresses will be messed up, but there is only one email adress in the entire dataset)*

*Looking foward to better solutions.*

*from Yang*

In [10]:
df = pd.read_csv("ner.csv") #import data
row_count = 0
for sentence in df["Sentence"]:
  df['Sentence'][row_count] = sentence.replace("?","#").replace("(","{").replace(")","}").replace("[","{").replace("]","}").replace("+","|").replace("$","@")
  row_count += 1

In [ ]:
#df = pd.read_csv("ner_modified.csv") #import data

training_data = []

sent_count = 0

for sentence in df["Sentence"]: #extract each sentence
  print("=======================================")
  #print(sent_count)
  #print(sentence)

  entities = []
  label = df.loc[sent_count,'Tag'].replace("'","").replace("[","").replace("]","").split(",") #modify the "Tag" column to lists
  token_list = sentence.split(" ") #find each token
  sentence += ' ' #this line is needed for code below.
  token_met_list = [] #tokens might repeat but we want each one's indices in the sentence.
  #I record each one so I know if they are repetitive.
  token_count = 0
  for token in token_list: #extract each token
    all_indices_for_this_token = [(i.start(), i.end()) for i in re.finditer(token+' ',sentence)]
    # This line use RE to identify start and end of each token.
    # I copied this from stackOverFlow. This line creates most of the errors and I don't know why!!!
    # I added a space after each token so things like "cat" will not mess up with "cater" or "concat".

    times_met_this_token = token_met_list.count(token)
    start = all_indices_for_this_token[times_met_this_token][0] # This is another line that often goes wrong.
    end = all_indices_for_this_token[times_met_this_token][1] - 1
    token_met_list.append(token) #mark the times we met this word
    entities.append((start,end,label[token_count])) # entities

    token_count += 1
    #print(token)

  training_data.append((sentence,{'entities':entities}))
  sent_count += 1

Streaming output truncated to the last 5000 lines.
6449
After an early December 2008 court ruling that dissolved the ruling PPP and two other coalition parties for election violations , the Democrat Party formed a new coalition government and ABHISIT Wetchachiwa became prime minister .
6450
In October 2008 THAKSIN fled abroad in advance of an abuse of power conviction and has agitated his followers from abroad since then .
6451
THAKSIN supporters under the banner of the United Front for Democracy Against Dictatorship { UDD , aka red-shirts } rioted in April 2009 , shutting down an ASEAN meeting in Pattaya .
6452
Following a February 2010 court verdict confiscating half of THAKSIN 's frozen assets , the UDD staged large protests between March and May 2010 , and occupied several blocks of downtown Bangkok .
6453
Clashes between security forces and protesters , elements of which were armed , resulted in at least 92 deaths and an estimated @ 1.5 billion in arson-related property losses .
6

In [11]:
pd.DataFrame(training_data).to_csv('training_data.csv', index=False)

In [14]:
df2 = pd.read_csv('training_data.csv')

In [15]:
df2.head()

,0,1
0,Thousands of demonstrators have marched throug...,"{'entities': [(0, 9, 'O'), (10, 12, ' O'), (13..."
1,Families of soldiers killed in the conflict jo...,"{'entities': [(0, 8, 'O'), (9, 11, ' O'), (12,..."
2,They marched from the Houses of Parliament to ...,"{'entities': [(0, 4, 'O'), (5, 12, ' O'), (13,..."
3,"Police put the number of marchers at 10,000 wh...","{'entities': [(0, 6, 'O'), (7, 10, ' O'), (11,..."
4,The protest comes on the eve of the annual con...,"{'entities': [(0, 3, 'O'), (4, 11, ' O'), (12,..."


In [ ]:
# This is just copied from the dc group chat
nlp = spacy.blank("en")
doc_bin = DocBin()
for training_example in tqdm(training_data):
  text = training_example['text'] # This line need to be revised. It doesn't work because there is no "text" column
  labels = training_example['entities']
  doc = nlp.make_doc(text)
  ents = []
  for start, end, label in labels:
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is None:
      print("Skipping entity")
    else:
      ents.append(span)
  filtered_ents = filter_spans(ents)
  doc.ents = filtered_ents
  doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

NameError: ignored